# Refactor notebook: gfedb_utils.py

In [51]:
import os
import sys

# Uncomment for Jupyter Notebook
for path in ['../','../src/']:
    sys.path.append(path) if path not in sys.path else ""

import logging
import re
import ast
import time
import urllib.request
from Bio import AlignIO
from Bio.SeqFeature import SeqFeature
from Bio.SeqRecord import SeqRecord
from seqann.models.annotation import Annotation
from Bio import SeqIO
from pyard import ARD
from seqann.gfe import GFE
from csv import DictWriter
from pathlib import Path
from constants import *
import hashlib

## Testing

### Global Variables

In [52]:
dbversion = "3420"
verbose = True
verbosity = 1
alignments = True
kir = False
imgt_release = f'{dbversion[0]}.{dbversion[1:3]}.{dbversion[3]}'

In [53]:
ard = ARD(dbversion)

In [54]:
gfe_maker = GFE(verbose=verbose, 
    verbosity=verbosity,
    load_features=False, 
    store_features=True,
    loci=hla_loci)

In [55]:
# Output memory profile to check for leaks
_mem_profile = True if '-p' in sys.argv else False

if _mem_profile:
    from pympler import tracker, muppy, summary
    tr = tracker.SummaryTracker()

### Refactored Functions

In [56]:
def seq_hasher(seq, n=32):
    """Takes a nucleotide or amino acid sequence and returns a reproducible
    integer UUID. Used to create shorter unique IDs since Neo4j cannot index 
    a full sequence. Can be also be used for any string."""

    m = hashlib.md5()
    m.update(seq)

    return str(int(m.hexdigest(), 16))[:n]

In [57]:
def hla_alignments(dbversion):
    gen_aln = {l: {} for l in hla_loci}
    nuc_aln = {l: {} for l in hla_loci}
    prot_aln = {l: {} for l in hla_loci}

    #logging.info(f'HLA alignments:\n{hla_align}')

    for loc in hla_align:
        msf_gen = ''.join([data_dir, dbversion, "/", loc.split("-")[1], "_gen.msf"])
        msf_nuc = ''.join([data_dir, dbversion, "/", loc.split("-")[1], "_nuc.msf"])
        msf_prot = ''.join([data_dir, dbversion, "/", loc.split("-")[1], "_prot.msf"])

        logging.info(f'Loading {"/".join(msf_gen.split("/")[-3:])}')
        align_gen = AlignIO.read(open(msf_gen), "msf")
        gen_seq = {"HLA-" + a.name: str(a.seq) for a in align_gen}
        del align_gen
        logging.info(f'{str(len(gen_seq))} genomic alignments loaded')
        gen_aln.update({loc: gen_seq})

        logging.info(f'Loading {"/".join(msf_nuc.split("/")[-3:])}')
        align_nuc = AlignIO.read(open(msf_nuc), "msf")
        nuc_seq = {"HLA-" + a.name: str(a.seq) for a in align_nuc}
        del align_nuc
        logging.info(f'{str(len(nuc_seq))} nucleotide alignments loaded')
        nuc_aln.update({loc: nuc_seq})

        # https://github.com/ANHIG/IMGTHLA/issues/158
        # if str(dbversion) == ["3320", "3360"]:
        #    continue

        logging.info(f'Loading {"/".join(msf_prot.split("/")[-3:])}')
        align_prot = AlignIO.read(open(msf_prot), "msf")
        prot_seq = {"HLA-" + a.name: str(a.seq) for a in align_prot}
        del align_prot
        logging.info(f'{str(len(prot_seq))} protein alignments loaded')
        prot_aln.update({loc: prot_seq})

    return gen_aln, nuc_aln, prot_aln

In [58]:
def get_features(seqrecord):
    j = 3 if len(seqrecord.features) > 3 else len(seqrecord.features)
    fiveutr = [["five_prime_UTR", SeqRecord(seq=seqrecord.features[i].extract(seqrecord.seq), id="1")] for i in
               range(0, j) if seqrecord.features[i].type != "source"
               and seqrecord.features[i].type != "CDS" and isinstance(seqrecord.features[i], SeqFeature)
               and not seqrecord.features[i].qualifiers]
    feats = [[''.join([str(feat.type), "_", str(feat.qualifiers['number'][0])]), SeqRecord(seq=feat.extract(seqrecord.seq), id="1")]
             for feat in seqrecord.features if feat.type != "source"
             and feat.type != "CDS" and isinstance(feat, SeqFeature)
             and 'number' in feat.qualifiers]

    threeutr = []
    if len(seqrecord.features) > 1:
        threeutr = [["three_prime_UTR", SeqRecord(seq=seqrecord.features[i].extract(seqrecord.seq), id="1")] for i in
                    range(len(seqrecord.features) - 1, len(seqrecord.features)) if
                    seqrecord.features[i].type != "source"
                    and seqrecord.features[i].type != "CDS" and isinstance(seqrecord.features[i], SeqFeature)
                    and not seqrecord.features[i].qualifiers]

    feat_list = fiveutr + feats + threeutr
    annotation = {k[0]: k[1] for k in feat_list}

    return annotation

In [59]:
# Returns base pair and amino acid sequences from CDS data
def get_cds(allele):

    feat_types = [f.type for f in allele.features]
    bp_seq = None
    aa_seq = None
    
    if "CDS" in feat_types:
        cds_features = allele.features[feat_types.index("CDS")]
        if 'translation' in cds_features.qualifiers:

            if cds_features.location is None:
                logging.info(f"No CDS location for feature in allele: {allele.name}")
            else:
                bp_seq = str(cds_features.extract(allele.seq))
                aa_seq = cds_features.qualifiers['translation'][0]
                
    return bp_seq, aa_seq

In [60]:
# Streams dictionaries as rows to a file
def append_dict_as_row(dict_row, file_path):

    try:
        header = list(dict_row.keys())

        # Check if file exists
        csv_file = Path(file_path)
        if not csv_file.is_file():

            # Create the file and add the header
            with open(file_path, 'a+', newline='') as write_obj:
                dict_writer = DictWriter(write_obj, fieldnames=header)
                dict_writer.writeheader()

        # Do not add an else statement or the first line will be skipped
        with open(file_path, 'a+', newline='') as write_obj:
            dict_writer = DictWriter(write_obj, fieldnames=header)
            dict_writer.writerow(dict_row)

        return
    except Exception as err:
        logging.error(f'Could not add row')
        raise err

In [61]:
# Outputs memory of objects during execution to sheck for memory leaks
if _mem_profile:
    def memory_profiler(mode='all'):

        # Print a summary of memory usage every n alleles
        all_objects = muppy.get_objects()
        sum2 = summary.summarize(all_objects)

        original_stdout = sys.stdout

        if mode == 'all' or mode == 'agg':
            with open("summary_agg.txt", "a+") as f:
                sys.stdout = f
                summary.print_(sum2)
                sys.stdout = original_stdout;

        if mode == 'all' or mode == 'diff':
            with open("summary_diff.txt", "a+") as f:
                sys.stdout = f
                tr.print_diff()
                sys.stdout = original_stdout;    

        return

In [62]:
def parse_dat(data_dir, dbversion):
    
    try:
        logging.info("Parsing DAT file...")
        dat_file = ''.join([data_dir, 'hla.', dbversion, ".dat"])
        
        return SeqIO.parse(dat_file, "imgt")
    
    except Exception as err:
        logging.error(f'Could not parse file: {dat_file}')
        raise err

In [63]:
def get_groups(allele):
    a_name = allele.description.split(",")[0].split("-")[1]
    groups = [["HLA-" + ard.redux(a_name, grp), grp] if ard.redux(a_name, grp) != a_name else None for
                grp in ard_groups]

    # expre_chars = ['N', 'Q', 'L', 'S']
    # to_second = lambda a: ":".join(a.split(":")[0:2]) + \
    #    list(a)[-1] if list(a)[-1] in expre_chars and \
    #    len(a.split(":")) > 2 else ":".join(a.split(":")[0:2])
    # seco = [[to_second(a_name), "2nd_FIELD"]]

    return groups #list(filter(None, groups)) # + seco # --> why filter(None, ...) ?

In [64]:
### Refactor builed gfes
def build_GFE(allele):
    
    # Build and stream the GFE rows
    try:
        _seq = str(allele.seq)

        gfe_row = {
            "gfe_name": gfe,
            "allele_id": allele.id,
            "locus": locus,
            "hla_name": hla_name,
            "a_name": hla_name.split("-")[1],
            "seq_id": seq_hasher(_seq.encode('utf-8')),
            "sequence": _seq,
            "length": len(_seq),
            "imgt_release": imgt_release
        }

        return gfe_row
               
    except Exception as err:
        logging.error(f'Failed to write GFE data for allele ID {allele.id}')
        raise err 

In [65]:
def build_feature(feature, allele):
    
    try:
        # features preprocessing steps
        # 1) Convert seqann type to python dict using literal_eval
        # 2) add GFE foreign keys: allele_id, hla_name
        # 3) calculate columns: length             

        # Append allele id's
        # Note: Some alleles may have the same feature, but it may not be the same rank, 
        # so a feature should be identified with its allele by allele_id or HLA name

        feature["gfe_name"] = gfe
        feature["term"] = feature["term"].upper()
        feature["allele_id"] = allele.id 
        feature["hla_name"] = hla_name
        feature["imgt_release"] = imgt_release

        # Avoid null values in CSV for Neo4j import
        feature["hash_code"] = "none" if not feature["hash_code"] else feature["hash_code"]

        return feature

    except Exception as err:
        logging.error(f'Failed to write feature for allele {allele.id}')
        logging.error(err)

In [66]:
def build_genomic_alignment(allele):

    if allele.description.split(",")[0] in gen_aln[locus]:
      
        try:
            
            aligned_gen = gen_aln[locus][allele.description.split(",")[0]]

            gen_align_row = {
                "label": "GEN_ALIGN",
                "seq_id": seq_hasher(aligned_gen.encode('utf-8')),
                "gfe_name": gfe,
                "hla_name": hla_name,
                "a_name": hla_name.split("-")[1],
                "length": len(aligned_gen),
                "rank": "0", # TO DO: confirm how this value is derived
                "bp_sequence": aligned_gen,
                "aa_sequence": "",
                "imgt_release": imgt_release # 3.24.0 instead of 3240
            }

            return gen_align_row
    
        except Exceptions as err:
            logging.error(f'Failed to write protein alignment for allele {allele.id}')
            raise err
    
    else:
        logging.info(f'No genomic alignments found')
        return

In [67]:
def build_nucleotide_alignment(allele):
    
    if allele.description.split(",")[0] in nuc_aln[locus]:

        try:
            aligned_nuc = nuc_aln[locus][allele.description.split(",")[0]]

            nuc_align_row = {
                "label": "NUC_ALIGN",
                "seq_id": seq_hasher(aligned_nuc.encode('utf-8')),
                "gfe_name": gfe,
                "hla_name": hla_name,
                "a_name": hla_name.split("-")[1],
                "length": len(aligned_nuc),
                "rank": "0", # TO DO: confirm how this value is derived
                "bp_sequence": aligned_nuc,
                "aa_sequence": "",
                "imgt_release": imgt_release
            }

            return nuc_align_row
        
        except Exceptions as err:
            logging.error(f'Failed to write protein alignment for allele {allele.id}')
            raise err
    
    else:
        logging.info(f'No nucleotide alignments found')
        return

In [68]:
def build_protein_alignment(allele):
    
    if allele.description.split(",")[0] in prot_aln[locus]:
        
        try:
            aligned_prot = prot_aln[locus][allele.description.split(",")[0]]

            prot_align_row = {
                "label": "PROT_ALIGN",
                "seq_id": seq_hasher(aligned_prot.encode('utf-8')),
                "gfe_name": gfe,
                "hla_name": hla_name,
                "a_name": hla_name.split("-")[1],
                "length": len(aligned_prot),
                "rank": "0", # TO DO: confirm how this value is derived
                "bp_sequence": "",
                "aa_sequence": aligned_prot,
                "imgt_release": imgt_release
            }

            return prot_align_row
        
        except Exceptions as err:
            logging.error(f'Failed to write protein alignment for allele {allele.id}')
            raise err
    
    else:
        logging.info(f'No protein alignments found')
        return

In [69]:
def build_group(group, allele):
    # Build and stream the ARD group rows
    try:
        group_row = {
            "gfe_name": gfe,
            "allele_id": allele.id,
            "hla_name": hla_name,
            "a_name": hla_name.split("-")[1],
            "ard_id": group[0],
            "ard_name": group[1],
            "locus": locus,
            "imgt_release": imgt_release
        }

        file_path = f'{data_dir}csv/all_groups.{dbversion}.csv'
        
        return group_row

    except Exception as err:
        logging.error(f'Failed to write groups for allele {allele.id}')
        raise err

In [70]:
def build_cds(allele):
    # Build and stream the CDS rows
    try:
        # Build CDS dict for CSV export, foreign key: allele_id, hla_name
        bp_seq, aa_seq = get_cds(allele)

        cds_row = {
            "gfe_name": gfe,
            # "gfe_sequence": str(allele.seq),
            # "allele_id": allele.id,
            # "hla_name": hla_name,
            "bp_seq_id": seq_hasher(bp_seq.encode('utf-8')),
            "bp_sequence": bp_seq,
            "aa_seq_id": seq_hasher(aa_seq.encode('utf-8')),
            "aa_sequence": aa_seq,
            # "imgt_release": imgt_release
        }

        return cds_row

    except Exception as err:
        logging.error(f'Failed to write CDS data for allele {allele.id}')
        raise err

In [76]:
def process_allele(allele):
    
    hla_name = allele.description.split(",")[0]

    complete_annotation = get_features(allele)

    ann = Annotation(annotation=complete_annotation,
            method='match',
            complete_annotation=True)

    # This process takes a long time
    logging.info(f"Getting GFE data for allele {allele.id}...")
    features, gfe = gfe_maker.get_gfe(ann, locus)

    # gfe_sequences
    file_name = ''.join([data_dir, f'csv/gfe_sequences.{dbversion}.csv'])
    gfe_row = build_GFE(allele)
    append_dict_as_row(gfe_row, file_name)

    # all_features
    # features contains list of seqann objects, converts to dict, destructive step
    features = \
        [ast.literal_eval(str(feature) \
            .replace('\'', '"') \
            .replace('\n', '')) \
            for feature in features]  

    file_path = f'{data_dir}csv/all_features.{dbversion}.csv'

    for feature in features:
        feature_row = build_feature(feature, allele)
        append_dict_as_row(feature_row, file_path)

    # all_alignments
    if alignments:
        file_path = f'{data_dir}csv/all_alignments.{dbversion}.csv'
        append_dict_as_row(build_genomic_alignment(allele), file_path)
        append_dict_as_row(build_nucleotide_alignment(allele), file_path)
        append_dict_as_row(build_protein_alignment(allele), file_path)

    # all_groups
    groups = get_groups(allele)

    file_path = f'{data_dir}csv/all_groups.{dbversion}.csv'

    for group in groups:
        group_row = build_group(group, allele)
        append_dict_as_row(group_row, file_path)

    # all_cds
    file_path = f'{data_dir}csv/all_cds.{dbversion}.csv'
    append_dict_as_row(build_cds(allele), file_path)
        
    return

### Allele Processing

In [77]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

logging.debug(f'args: {sys.argv}')

In [73]:
alleles = parse_dat(data_dir, "3420")

2021-04-25 02:39:18 - root - INFO - Parsing DAT file...


In [34]:
# Load alignments data
if alignments:
    gen_aln, nuc_aln, prot_aln = hla_alignments(dbversion)

2021-04-25 02:35:17 - root - INFO - Loading data/3420/A_gen.msf
2021-04-25 02:35:18 - root - INFO - 3067 genomic alignments loaded
2021-04-25 02:35:18 - root - INFO - Loading data/3420/A_nuc.msf
2021-04-25 02:35:18 - root - INFO - 6291 nucleotide alignments loaded
2021-04-25 02:35:18 - root - INFO - Loading data/3420/A_prot.msf
2021-04-25 02:35:19 - root - INFO - 6291 protein alignments loaded
2021-04-25 02:35:19 - root - INFO - Loading data/3420/B_gen.msf
2021-04-25 02:35:19 - root - INFO - 3643 genomic alignments loaded
2021-04-25 02:35:19 - root - INFO - Loading data/3420/B_nuc.msf
2021-04-25 02:35:20 - root - INFO - 7561 nucleotide alignments loaded
2021-04-25 02:35:20 - root - INFO - Loading data/3420/B_prot.msf
2021-04-25 02:35:21 - root - INFO - 7561 protein alignments loaded
2021-04-25 02:35:21 - root - INFO - Loading data/3420/C_gen.msf
2021-04-25 02:35:21 - root - INFO - 3508 genomic alignments loaded
2021-04-25 02:35:21 - root - INFO - Loading data/3420/C_nuc.msf
2021-04-25 

In [79]:
for allele in alleles:
    
    locus = allele.description.split(",")[0].split("*")[0]
    
    allele_can_be_processed = \
        hasattr(allele, 'seq') and \
        (locus in hla_loci or locus == "DRB5") and \
        (len(str(allele.seq)) > 5)
    
    if allele_can_be_processed:
        process_allele(allele)
    else:
        logger.warn(f'Did not process allele: {hla_name} for locus {locus}')

    break

2021-04-25 02:40:16 - root - INFO - Getting GFE data for allele HLA15760.1...
2021-04-25 02:40:17 - Logger.seqann.gfe - INFO - GFE = HLA-Aw46-1-1-1-1-1-90-1-1-1-1-1-1-1-1-1-18


In [ ]:
# TO DO - Conver these to generators
gen_aln, nuc_aln, prot_aln